# Lasso Example

In [1]:
import PyPlot
import ProximalOPT
import HD
using ArrayViews

INFO: Loading help data...


In [26]:
reload("ProximalOPT")

Generate Data

In [2]:
srand(1)

n = 500
p = 2500

t_beta = sprandn(p,1,0.05) 
t_beta = vec(full(t_beta))
A = randn(n,p)
A = A ./ mapslices(norm, A, 1)
v = sqrt(0.001)*randn(n)
b = A*t_beta + v

println("solving instance with $n examples, $p variables")
println("nnz(x0) = $(countnz(t_beta)); signal-to-noise ratio: $(norm(A*t_beta)^2/norm(v)^2)")

AA = A'A 
Ab = A'b 
Abn = -A'b

gamma_max = norm(A'*b, Inf)
gamma = 0.1*gamma_max;

solving instance with 500 examples, 2500 variables
nnz(x0) = 129; signal-to-noise ratio: 251.22786125240205


Proximal gradient

In [29]:
h_beta = zeros(p)

tic()
f = ProximalOPT.quad_f(view(AA, :, :), view(Abn, :))
g = ProximalOPT.prox_l1(gamma)
tr = ProximalOPT.prox_grad!(h_beta, f, g; beta=0.9, show_trace = false, ABSTOL=1e-4)
prox_res = (copy(h_beta), tr, toc());

elapsed time: 0.728625665 seconds (99800 bytes allocated)
elapsed time: 0.731257561 seconds


Fast proximal gradient

In [32]:
h_beta1 = zeros(p)

tic()
f = ProximalOPT.quad_f(view(AA, :, :), view(Abn, :))
g = ProximalOPT.prox_l1(gamma)
tr = ProximalOPT.acc_prox_grad!(h_beta1, f, g; beta=0.9, show_trace = false, ABSTOL=1e-4)
acc_prox_res = (copy(h_beta1), tr, toc());

elapsed time: 0.667481345 seconds


Block coordinate descent

In [12]:
h_beta2 = spzeros(p,1)

tic()
lambda = gamma * ones(p)
HD.lasso!(h_beta2, AA, Ab, lambda)
h_beta2 = vec(full(h_beta2))
coor_descent = (copy(h_beta2), toc());

elapsed time: 0.46685419 seconds


In [9]:
maximum(abs(h_beta1 - h_beta2))

0.017706480319313433

In [10]:
obj_val(x) = dot(x, AA*x) / 2. - dot(x, Ab) + gamma * norm(x, 1) + dot(b, b) / 2.

obj_val (generic function with 1 method)

In [13]:
obj_val(h_beta), obj_val(h_beta1), obj_val(h_beta2)

(26.295687940948326,26.296337812515745,26.295685167540775)